In [1]:
from pyscipopt import Model, quicksum, multidict
from cflp_function import *
from calculate_od import *

c:\Users\User\anaconda3\envs\scip\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
folder_path = "app_data"
I = load_data_from_pickle(folder_path, 'Farm_test.pickle')
d = load_data_from_pickle(folder_path, 'manure_production_test.pickle')

In [3]:
sum(d.values())

704905

In [4]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df
def load_gdf(gdf_path):
    gdf = gpd.read_file(gdf_path)
    return gdf

In [5]:
main_crs = "EPSG:4326"

In [6]:
### LOAD DATA ###
loi = load_csv('./hex/loi.csv') # Mock candidate sites
farm_gdf = load_gdf("./farm/farm_new.shp")
n = load_gdf("./osm_network/G_n.shp") # Road network nodes
n = n.to_crs(main_crs)
### CALCULATE OD MATRIX ###
loi.index = range(1, len(loi) +1) # Reset index to start with 1 (because users don't like 0 as the 1st value...)
loi_gdf = loi_to_gdf(loi) # Find centroid of hexagons and convert to gdf
loi_gdf['y'] = loi_gdf['geometry'].y
loi_gdf['x'] = loi_gdf['geometry'].x
find_closest_osmid(farm_gdf, n)
find_closest_osmid(loi_gdf, n)
c, plant = calculate_od_matrix(farm_gdf, loi_gdf, cost_per_km=0.69)

In [7]:
M, f = assign_capacity_capex(plant)

In [8]:
J = plant

In [9]:
p=0.1

In [10]:
model = Model("flp_percentage_demand")

x, y, z = {}, {}, {}
total_demand = sum(d[i] for i in I)
target_demand = total_demand * p

for j in J:
    y[j] = model.addVar(vtype="B", name="y(%s)" % j)
    for i in I:
        x[i, j] = model.addVar(vtype="C", name="x(%s,%s)" % (i, j))
        z[i, j] = model.addVar(vtype="B", name="z(%s,%s)" % (i, j))

for i in I:
    model.addCons(quicksum(x[i, j] for j in J) == d[i] * z[i, j], "Demand(%s)" % i)

for j in M:
    model.addCons(quicksum(x[i, j] for i in I) <= M[j] * y[j], "Capacity(%s)" % j)

for (i, j) in x:
    model.addCons(x[i, j] <= d[i] * y[j], "Strong(%s,%s)" % (i, j))

model.addCons(quicksum(x[i, j] for i in I for j in J) >= target_demand, "PercentageDemand")

model.setObjective(
    quicksum(f[j] * y[j] for j in J) +
    quicksum(c[i, j] * x[i, j] for i in I for j in J),
    "minimize"
)

model.data = x, y, z

In [11]:
model.optimize()

In [12]:
EPS = 1.e-6
x,y,z = model.data
assignment = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]

In [14]:
total_cost = model.getObjVal()

# Create a dictionary to store the results
result_dict = {f: [] for f in facilities}
# Iterate over edges and populate the result_dict
for (i, j) in assignment:
    if j in facilities:
        result_dict[j].append(i)

# Get percentage of utilization
x_values = {(i, j): model.getVal(x[i, j]) for (i, j) in x if model.getVal(x[i, j]) > EPS} # get how much is flowing between every assignment
flow_matrix = np.array([[x_values.get((i, j), 0) for j in J] for i in I]) # create a flow matrix (len(farm)xlen(plant))
column_sum = np.sum(flow_matrix, axis=0) # sum of total flow going to every plant
used_capacity = (column_sum/np.array(list(M.values())))*100

In [19]:
used_capacity_df = pd.DataFrame(used_capacity, index=J)

In [20]:
used_capacity_df

,0
1,0.000000
2,0.000000
3,0.000000
4,59.060453
5,0.000000
6,0.000000
7,0.000000


In [26]:
# Example DataFrames
farm_df = pd.DataFrame({
    'color': ['red', 'blue', 'green', 'yellow'],
    # ... other columns in farm_df
})

digester_df = pd.DataFrame({
    'color': [
        [255, 165, 0],  # orange
        [128, 0, 128],  # purple
        [165, 42, 42],  # brown
        [255, 192, 203]  # pink
    ],
    # ... other columns in digester_df
})

assignment_decision = {
    0: [1, 2],
    1: [0, 3],
    # ... other keys and values in assignment_decision
}


In [29]:
# Update farm_df based on assignment_decision
for digester_index, farm_indices in assignment_decision.items():
    digester_color = digester_df.loc[digester_index, 'color']
    print(farm_indices)
    farm_df.loc[farm_indices, 'color'] = str(digester_color)

# Now, farm_df has been updated based on digester_df and assignment_decision
print(farm_df)

[1, 2]
[0, 3]
           color
0  [128, 0, 128]
1  [255, 165, 0]
2  [255, 165, 0]
3  [128, 0, 128]


In [24]:
digester_df

,color
0,orange
1,purple
2,brown
3,pink


In [30]:
farm_df

,color
0,"[128, 0, 128]"
1,"[255, 165, 0]"
2,"[255, 165, 0]"
3,"[128, 0, 128]"
